In [ ]:
import pandas as pd
from Models import GNB

train_filepath = 'data/iris_train_set.csv'
test_filepath = 'data/iris_test_set.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)



In [ ]:
ytrain = train_data['iris_type']
Xtrain = train_data.drop(['iris_type'], axis=1)
ytest = test_data['iris_type']
Xtest = test_data.drop(['iris_type'], axis=1)

print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

In [ ]:
from preprocessing import Scaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler1_ = MinMaxScaler().fit_transform(Xtrain)
scaler1_ = pd.DataFrame(scaler1_, columns = Xtrain.columns)
scaler1_

In [ ]:
scaler1 = Scaler('minmax').fit_transform(Xtrain)
scaler1

In [ ]:
for i in range(scaler1.shape[0]):
    for j in range(scaler1.shape[1]):

        if round(scaler1.iloc[i,j],10) != round(scaler1_.iloc[i,j],10) :
            print(i,j)

In [ ]:
scaler2 = Scaler('t-distribution').fit_transform(Xtrain)
scaler2

In [ ]:
for col in Xtrain.columns:
    print(scaler1[col].max(), scaler1[col].min())
    print(scaler1_[col].max(), scaler1_[col].min())
# for col in Xtrain.columns:
#     print(scaler2[col].mean(), scaler2[col].std())

In [ ]:
gnb = GNB()
gnb.fit(Xtrain, ytrain)

In [ ]:
ytrain[ytrain == 0].shape[0]/ytrain.shape[0]

In [ ]:
print(gnb.cls_list)
print(gnb.cls_priors)

In [ ]:
print("---------------------mean----------------------")
print(gnb.mean_likelihood)
print("---------------------std----------------------")
print(gnb.std_likelihood)

In [ ]:
ypred = gnb.predict(Xtest)

In [ ]:
gnb.calc_accuracy(ypred, ytest)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
clf.fit(Xtrain, ytrain)
y_pred2 = clf.predict(Xtrain)

In [ ]:
gnb.calc_accuracy(y_pred2, ytrain)

In [ ]:
from Models import KNN
knn = KNN(3,'euclidean','t-disribution')

In [ ]:
knn.fit(Xtrain, ytrain)

In [ ]:
pred = knn.predict(Xtest)

In [ ]:
knn.calc_accuracy(ypred, ytest)

In [ ]:
from Models import KMeans
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
centers = 5

X_train, true_labels = make_blobs(n_samples=100, centers=centers, random_state=42)
X_train = StandardScaler().fit_transform(X_train)
sns.scatterplot(x=[X[0] for X in X_train],
                y=[X[1] for X in X_train],
                hue=true_labels,
                palette="deep",
                legend=None
                )
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=centers, init='kmeans++')
kmeans.fit(X_train)
# View results
class_centers, classification = kmeans.evaluate()
sns.scatterplot(x=[X[0] for X in X_train],
                y=[X[1] for X in X_train],
                hue=true_labels,
                style=classification,
                palette="deep",
                legend=None
                )
plt.plot([x for x, _ in kmeans.centroids],
         [y for _, y in kmeans.centroids],
         '+',
         markersize=10,
         )
plt.show()

In [ ]:
import numpy as np
a = [1,2,3,5]
np.not_equal(a,a).any()

In [ ]:
import numpy as np
from Models import Perceptron
inputs = [1,10,5,46,9]
weights = np.random.rand(5)
bias = np.random.randint(0,10)
print(inputs, weights, bias)

In [ ]:
p = Perceptron(inputs, weights, bias, "sign", learning_const=0.1)
str(p)

In [ ]:
p.gradient_descent(0)
p.weights

In [ ]:
print(p)

In [ ]:
import numpy as np
a = np.random.rand(5)
a

In [ ]:
(b *a).sum()

In [ ]:
print(a, b)
a = a+ 0*b
print(a, b)


In [ ]:
random_weights = np.random.rand(5)
random_weights = [x if np.random.randint(4)!=2 else -x for x in random_weights]
random_weights

In [ ]:
a = np.array([1,2, None])
print(a.size)
print(a.all() == np.array(None))
print(a.shape)

In [1]:
first = 2
second = 3
from Models import ANN

a = ANN((3,), "sigmoid")

import pandas as pd

y = pd.Series([1, 2, 2, 3])
X = pd.DataFrame(
    {
        "a": [0.35, 1.7, -1, 0.2],
        "b": [1, 5, 8, 1.6],
        "c": [1.2, 5, -1.4, -3.6],
        "d": [1, 5, 8, 1.6],
        "e": [1.2, 5, -1.4, -3.6],
    }
)


In [2]:
a.fit(X,y)
print(a)

[5, 3, 3]
delta_weight:[0. 0. 0. 0. 0.]
 prev_delta_weights:[1. 1. 1. 1. 1.]
weights:[ 0.8758214   0.24183712  0.51601535 -0.68247705  0.68986714]
----
delta_weight:[2.76781163e-05 7.90803322e-05 9.48963987e-05 7.90803322e-05
 9.48963987e-05]
 prev_delta_weights:[0. 0. 0. 0. 0.]
weights:[ 0.87584908  0.2419162   0.51611024 -0.68239797  0.68996204]
----
delta_weight:[0. 0. 0. 0. 0.]
 prev_delta_weights:[1. 1. 1. 1. 1.]
weights:[-0.80873156 -0.35814318 -0.13170146  0.60585652  0.84123376]
----
delta_weight:[0.00021459 0.00061311 0.00073574 0.00061311 0.00073574]
 prev_delta_weights:[0. 0. 0. 0. 0.]
weights:[-0.80851697 -0.35753006 -0.13096572  0.60646964  0.8419695 ]
----
delta_weight:[0. 0. 0. 0. 0.]
 prev_delta_weights:[1. 1. 1. 1. 1.]
weights:[-0.19690737  0.29030027 -0.69562619 -0.04800579 -0.66946714]
----
delta_weight:[0.00043768 0.00125053 0.00150063 0.00125053 0.00150063]
 prev_delta_weights:[0. 0. 0. 0. 0.]
weights:[-0.19646969  0.2915508  -0.69412555 -0.04675527 -0.6679665 ]
--

IndexError: list index out of range

In [ ]:
a.network[0][0].weights

In [ ]:
import numpy as np
np.array([[1]*6]*3)

In [ ]:
a.output_layer

In [ ]:
a.output_layer

In [ ]:
import numpy as np
import pandas as pd
X = pd.Series(['a','b','a','a'])
X

In [ ]:
X.value_counts()['a']

In [ ]:
-np.log2(0.5)*0.5*2

In [ ]:
def calc_entropy(X :pd.Series):
    val_count = X.value_counts()
    entropy = 0
    for val, count in val_count.items():
        prob = count / X.shape[0]
        entropy -= prob * np.log2(prob)
    return entropy

In [ ]:
a = ['young', 'young', 'middle', 'young', 'middle', 'old', 'young','old']
b = ['yes','yes', 'yes','no','yes','no','no','no']
X = pd.Series(a)
Y = pd.Series(b)

In [ ]:
print("Entropy of X:", calc_entropy(X))
print("Entropy of Y:", calc_entropy(Y))

In [ ]:
def specific_conditional_entropy(X :pd.DataFrame ,entropy_col, second_col, val):
        return calc_entropy(X[X[second_col]==val][entropy_col])

In [ ]:
df = pd.DataFrame({'X':X,'Y':Y})


print(specific_conditional_entropy(df,'X','Y', 'yes'))
print(specific_conditional_entropy(df,'X','Y', 'no'))


In [ ]:
def conditional_entropy(df,x,y):
    entropy = 0
    for val, count in df[y].value_counts().items():
        prob = count / df.shape[0]
        entropy += prob * specific_conditional_entropy(df, x,y,val)
    return entropy

In [ ]:
conditional_entropy(df,'Y','X')

In [ ]:
df = pd.DataFrame({'x1':[1,0,1,0],'x2':[1,0,0,0], 'Y': ['pos','pos','neg','neg']})
df

In [ ]:
from Models import Perceptron, ANN
a = ANN((1,0), "sigmoid")
p = Perceptron(activation_func="sigmoid")
print(p)

In [ ]:
p.set_weights(a.generate_init_weights(5))
print(p)

In [ ]:
p.set_inputs([1,4,1,5,6])
print(p)

In [ ]:
p.fit([0,1,1,0,0])

In [ ]:
conditional_entropy(df,'Y','x2')